**Let us read ORC**

Книжка с разбором ORC файла, предыстория и смысл - см блог (последняя ссылка ниже).

Ссылки:

* spec: https://orc.apache.org/specification/ORCv1/
* protocol buffers: https://developers.google.com/protocol-buffers/docs/pythontutorial
* orc proto: https://github.com/apache/hive/blob/trunk/ql/src/protobuf/org/apache/hadoop/hive/ql/io/orc/orc_proto.proto
* apache orc: https://github.com/apache/orc
* блог про это: http://wiki.alfastrah.ru/pages/viewpage.action?pageId=97983312

In [243]:
import zlib # части файла могут быть сжаты...

In [244]:
import orc_proto_pb2 # без этого - никуда, все метаданные хранятся в protobuf... пришлось сгенерить по github-у выше

В качестве примера взят код с сайта проекта (https://orc.apache.org/docs/core-cpp.html), прямо из примера была создана "табличка" 

`struct<x:int,y:int>`

и заполнена таким нехитрым образом

`
for (i = 0; i < 10000; ++i) { 
    x->data[rows] = i; 
    y->data[rows] = i * 3;
}`

Результат просто (`writer->add(*batch)`) засунули в батчи размером 1024 строк и записали в файл, который дальше будет читать и изучать.

In [245]:
with open("my-file.orc","rb") as f:
    buf = f.read()
len(buf)

628

Пока все бьется - размер файла 628 байт (как и обещали в блоге). И это - забегая вперед - безо всякой компрессии!

Что дальше: как известно из спецификации, чтение начинается с хвоста: читаем PostScript. 

Для этого сначала считаем последний байт файла - он содержит размер PostScript-а, который (единственный из всех частей файла) никогда не сжимается.

**Часть 1: PostScript и Footer**

считаем и распечатаем (protobuf предоставляет такую замечательную возможность - супер просто...)

**Шаг 1: Postdcript**

In [246]:
psLen = buf[-1] # psLen
print("psLen",psLen)

psLen 23


In [247]:
psBuf = buf[-psLen-1:-1]

In [248]:
pScript = orc_proto_pb2.PostScript()
pScript.ParseFromString(psBuf)
print(pScript)
if pScript.compression not in [0,1]: # [NONE,ZLIB]
    print("Compression unknown")
comprOffs = 3 if pScript.compression==1 else 0

footerLength: 115
compression: NONE
compressionBlockSize: 65536
version: 0
version: 12
metadataLength: 50
writerVersion: 4
magic: "ORC"



Что мы здесь видим:

* файл не сжат (ZLIB я отключил в опциях создания файла)
* размер футера и метаданных (будем использовать ниже)
* версия "писателя": важно (!). в библиотеке стояло 6 (текущая на момент ее клонирования), в результате все падало: HIVE не знал, что "бывают версии старше 4". Пришлось код поправить...

Остальное - просто так, для информации. Про сжатие - см. комментарии в блоге.

**Шаг 2: Footer**

дальше читаем footer (обратите внимание на обработку сжатых файлов, хоть у нас и не сжат, но код рассчитан на все случаи)

In [186]:
ftBuf = buf[-psLen-pScript.footerLength-1+comprOffs:-psLen-1]

In [187]:
footer = orc_proto_pb2.Footer()
if pScript.compression==1: ftBuf = zlib.decompress(ftBuf,-zlib.MAX_WBITS)# inflate version!
footer.ParseFromString(ftBuf)
print(footer)

headerLength: 3
contentLength: 436
stripes {
  offset: 3
  indexLength: 73
  dataLength: 276
  footerLength: 87
  numberOfRows: 10000
}
types {
  kind: STRUCT
  subtypes: 1
  subtypes: 2
  fieldNames: "x"
  fieldNames: "y"
  maximumLength: 0
  precision: 0
  scale: 0
}
types {
  kind: INT
  maximumLength: 0
  precision: 0
  scale: 0
}
types {
  kind: INT
  maximumLength: 0
  precision: 0
  scale: 0
}
numberOfRows: 10000
statistics {
  numberOfValues: 10000
  hasNull: false
}
statistics {
  numberOfValues: 10000
  intStatistics {
    minimum: 0
    maximum: 9999
    sum: 49995000
  }
  hasNull: false
}
statistics {
  numberOfValues: 10000
  intStatistics {
    minimum: 0
    maximum: 29997
    sum: 149985000
  }
  hasNull: false
}
rowIndexStride: 10000



Что интересного видим здесь

* есть размеры всех частей (лучше смотреть код ниже и описание структуры в блоге)
* страйп у нас получился 1
* структура таблицы
  * на верхнем уровне всегда struct
  * далее - две колонки (x и y) целого типа
* файловая статистика
  * 10 000 строк
  * по колоночно (нет нулей, мин, макс и сумма для каждой колонки)
* последнее - размер блока строк, на которые в страйпе будет создан индекс (10 тыс, значение по умолчанию)

**Шаг 3: Metadata**

далее читаем метаданные

In [188]:
metaBuf = buf[-psLen-pScript.footerLength-pScript.metadataLength-1+comprOffs:-psLen-pScript.footerLength-1]

In [189]:
meta = orc_proto_pb2.Metadata()
if pScript.compression==1: metaBuf = zlib.decompress(metaBuf,-zlib.MAX_WBITS)# inflate version!
meta.ParseFromString(metaBuf)
print(meta)

stripeStats {
  colStats {
    numberOfValues: 10000
    hasNull: false
  }
  colStats {
    numberOfValues: 10000
    intStatistics {
      minimum: 0
      maximum: 9999
      sum: 49995000
    }
    hasNull: false
  }
  colStats {
    numberOfValues: 10000
    intStatistics {
      minimum: 0
      maximum: 29997
      sum: 149985000
    }
    hasNull: false
  }
}



Что интересного в метаданных

статистика по каждому страйпу - у нас он один, поэтому только одна секция, значения совпадают со статистикой по файлу (а тип данных для статистик один - ColumnStatistics).

**Часть 2: данные**

**Шаг 1: Stripe Footer**

Он уже содержится в страйпе (который у нас один)

In [190]:
st = footer.stripes[0].offset+footer.stripes[0].indexLength+footer.stripes[0].dataLength
fn = st + footer.stripes[0].footerLength
stripeFtBuf = buf[st:fn]

In [191]:
stripeFooter = orc_proto_pb2.StripeFooter()
stripeFooter.ParseFromString(stripeFtBuf)
print(stripeFooter)

streams {
  kind: ROW_INDEX
  column: 0
  length: 14
}
streams {
  kind: ROW_INDEX
  column: 1
  length: 29
}
streams {
  kind: ROW_INDEX
  column: 2
  length: 30
}
streams {
  kind: PRESENT
  column: 0
  length: 20
}
streams {
  kind: PRESENT
  column: 1
  length: 20
}
streams {
  kind: DATA
  column: 1
  length: 103
}
streams {
  kind: PRESENT
  column: 2
  length: 20
}
streams {
  kind: DATA
  column: 2
  length: 113
}
columns {
  kind: DIRECT
  dictionarySize: 0
}
columns {
  kind: DIRECT_V2
  dictionarySize: 0
}
columns {
  kind: DIRECT_V2
  dictionarySize: 0
}
writerTimezone: "GMT"



Что мы здесь видим:

* сначала в страйпе идут три стрима с индексами
* далее идет PRESENT стрим для "псевдо колонки" (структуры)
* по PRESENT + DATA стримы для реальных колонок
* информация о кодировании колонок (DIRECT для структуры и DIRECT_V2 для реальных колонок)

Из непонятного пока - почему появились PRESENT стримы - вроде вверху видно, что "нулей" итак нет...

**Шаг 2: Индексы**

посмотрим на наиболее интересные - по псевдо колонке (он первый)

In [192]:
st = footer.stripes[0].offset
fn = st + stripeFooter.streams[0].length
stripeFtBuf = buf[st:fn]

In [193]:
rowIndex = orc_proto_pb2.RowIndex()
rowIndex.ParseFromString(stripeFtBuf)
print(rowIndex)

entry {
  positions: 0
  positions: 0
  positions: 0
  statistics {
    numberOfValues: 10000
    hasNull: false
  }
}



Пока непонятно... там и данных-то нет, попробуем посмотреть на реальную колонку

In [196]:
st = footer.stripes[0].offset + stripeFooter.streams[0].length
fn = st + stripeFooter.streams[1].length
stripeFtBuf = buf[st:fn]

In [197]:
rowIndex = orc_proto_pb2.RowIndex()
rowIndex.ParseFromString(stripeFtBuf)
print(rowIndex)

entry {
  positions: 0
  positions: 0
  positions: 0
  positions: 0
  positions: 0
  statistics {
    numberOfValues: 10000
    intStatistics {
      minimum: 0
      maximum: 9999
      sum: 49995000
    }
    hasNull: false
  }
}



Понятнее не стало... Смотрим данные

**Шаг 3: Data**

Смотрим PRESENT stream для структуры (по идее должен быть из единичек - все не нули)

In [200]:
st = footer.stripes[0].offset + footer.stripes[0].indexLength
fn = st + stripeFooter.streams[3].length
dataBuf = buf[st:fn]

In [209]:
i = 0
valCnt = 0
while True:
    hdr = dataBuf[i]
    valCnt += hdr
    if hdr<=127:
        data = dataBuf[i+1]
        print("HDR:",hdr,", DATA:", data)
    i += 2
    if i==20:
        break
print ("Values count:", valCnt)

HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 127 , DATA: 255
HDR: 77 , DATA: 255
Values count: 1220


у нас 10тыс строк, они бьются на run-ы по 127: 10000/127 = 78 и еще останется 94

Но, судя по данным (в стриме - 20 байт), что-то со значениями не совсем так.. Получилось 1220 значений

Не будем заморачиваться другими PRESENT стримами - похоже, они такие же, надо поскорее на данные посмотреть

**Колонка 1** данные

In [223]:
st = footer.stripes[0].offset + footer.stripes[0].indexLength + stripeFooter.streams[3].length + stripeFooter.streams[4].length
fn = st + stripeFooter.streams[5].length
dataBuf = buf[st:fn]

Первая колонка закодирована при помощи DIRECT_V2 для целых - это RLE v2

первые два бита - тип кодировки, дальше - зависит от типа

* 0=short: заголовок 1 байт
* 1=direct: 2 байта
* 2=patched base: 4 байта
* 3=delta: 2 байта

смотрим данные

In [233]:
print("HDR:",bin(dataBuf[0]),bin(dataBuf[1]))
print(dataBuf[2])
print(dataBuf[3])
print("HDR:",bin(dataBuf[4]),bin(dataBuf[5]))
print(bin(dataBuf[6]),bin(dataBuf[7]))
print(dataBuf[8])
print("HDR:",bin(dataBuf[9]),bin(dataBuf[10]))
print(bin(dataBuf[11]),bin(dataBuf[12]))
print(dataBuf[13])

HDR: 0b11000001 0b11111111
0
2
HDR: 0b11000001 0b11111111
0b10000000 0b1000
2
HDR: 0b11000001 0b11111111
0b10000000 0b10000
2


**Разбираем**

у нас получается delta кодировка (логично - последовательность)

первая пачка:
* 511 значений ширины 0 (т.е. никаких больше дельт не будет!, только первая)
* base = 0 (с базой получается как раз 512 значений)
* delta = 1
предполагаю, что должно обозначать 512 значений, начиная с нуля, увеличивающееся на 1 до 511 включительно...

вторая пачка (аналогично):
* 511 значений
* base = 1024 (?) предполагаю, что это есть 512 из-за зигзага...
* delta = 1
третья пачка (аналогично):
* 511 значений
* base = 1024 
* delta = 1
...

что получается - растет длина базы (за счет varint - кодируем целые). Остальное - постоянно. 

Посмотрим общую длину

всего 103 байта:
* 4 первая
* 25 еще пять (по 5 байт каждая)
* 84 еще 14 (по шесть байт каждая)
сошлось...

**Колонка 2** данные

In [241]:
st = footer.stripes[0].offset + footer.stripes[0].indexLength + stripeFooter.streams[3].length + \
    stripeFooter.streams[4].length + stripeFooter.streams[5].length + stripeFooter.streams[6].length
fn = st + stripeFooter.streams[7].length
dataBuf = buf[st:fn]

In [242]:
print("HDR:",bin(dataBuf[0]),bin(dataBuf[1]))
print(dataBuf[2])
print(dataBuf[3])
print("HDR:",bin(dataBuf[4]),bin(dataBuf[5]))
print(bin(dataBuf[6]),bin(dataBuf[7]))
print(dataBuf[8])
print("HDR:",bin(dataBuf[9]),bin(dataBuf[10]))
print(bin(dataBuf[11]),bin(dataBuf[12]))
print(dataBuf[13])

HDR: 0b11000001 0b11111111
0
6
HDR: 0b11000001 0b11111111
0b10000000 0b11000
6
HDR: 0b11000001 0b11111111
0b10000000 0b110000
6


**Разбираем**

у нас получается delta кодировка (логично - последовательность)

первая пачка:
* 511 значений ширины 0 (т.е. никаких больше дельт не будет!, только первая)
* base = 0 (с базой получается как раз 512 значений)
* delta = 3
предполагаю, что должно обозначать 512 значений, начиная с нуля, увеличивающееся на 3 до ... включительно...

в-общем все похоже на предыдущий вариант, только разница будет в длинах базы (она быстрее растет - макс 29997)

---------------------------------------------------------------------дальше всякая отладка и другие разности...

In [239]:
14*6


84

In [69]:
#with open("ft.gz","wb") as ff:
##    ff.write(ftBuf)
#len(ftBuf)
st = 2
ln = 2
"123456"[st:st+ln]

'34'

In [ ]:
#buf[-psLen-pScript.footerLength-1+2]
#f = open("footer.dat","rb")
#tBuf = f.read()
#.close()
#len(ftBuf)